In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import pandas as pd
from sodapy import Socrata

In [ ]:
#export
def download_file_maps(route = "./local/data/", filename = "colombia.geo.json"):
    
    gdd.download_file_from_google_drive(file_id='1wEiDe2B_ojoaLUnV3Ik67t_YpAC2nbPA',
                                        dest_path=route+filename,
                                        unzip=False, showsize=True, overwrite = True)
    return 

In [ ]:
download_file_maps()

In [ ]:
#export
def download_files_raw_dengue(route = "./local/data/"):
    gdd.download_file_from_google_drive(file_id='1xHXshzID1MBrKZKVvnc6JeO_ZU04HvQ_',
                                        dest_path=route+"dengue2007-2017.csv",
                                        unzip=False, showsize=True, overwrite = True)
    return
    

In [ ]:
download_files_raw_dengue()

65.2 MiB                                                                                                                                                                                                                                                                                                                                                           Done.


In [ ]:
def download_files_processed_dengue(route = "./local/data/"):
    gdd.download_file_from_google_drive(file_id='1p8cGQJV4yAhJRiSdPQ5z5fg3rRSgd-hu',
                                        dest_path=route+"dengue_weekly_2007-2020.csv",
                                        unzip=False, showsize=True, overwrite = True)
    return

In [ ]:
download_files_processed_dengue()

28.4 MiB                                                              Done.


In [ ]:
#export
def download_file_population(route = "./local/data/", filename = "population.csv"):
    
    gdd.download_file_from_google_drive(file_id='1RU2C4DoAubhXEhfVYLOtL4Yjg1LPERIc',
                                        dest_path=route+filename,
                                        unzip=False, showsize=True, overwrite = True)
    return 

In [ ]:
download_file_population()

640.0 KiB Done.


In [ ]:
def download_file_departments(route = "./local/data/", filename = "departments.csv"):
    
    gdd.download_file_from_google_drive(file_id='1ImNzdFwQF91aA4qE_7XKpOU7AFK1f7WS',
                                        dest_path=route+filename,
                                        unzip=False, showsize=True, overwrite = True)
    return 

In [ ]:
download_file_departments()

32.0 KiB Done.


In [ ]:
#pendientes todo lo del niño


# CLIMATE DATA DOWNLOAD

In [ ]:
setup = "repo" #alternativo: local
if setup == "repo":
    data_folder = "./local/data/"
else:
    data_folder = "./data/"

In [ ]:
def get_stations_from_municipality(client, dataset, municipio):
    results = client.get(dataset, limit = 1000, select = "distinct CodigoEstacion, CodigoSensor, NombreEstacion, Latitud, Longitud, DescripcionSensor, UnidadMedida", municipio=municipio)
    results_df = pd.DataFrame.from_records(results)
    return results_df

def get_time_series_from_station(client, dataset, municipio, codigoestacion, limit = 1000000):
    results = client.get(dataset, limit = limit, select = "*", municipio=municipio, codigoestacion = codigoestacion, order='fechaobservacion DESC')
    results_df = pd.DataFrame.from_records(results)
    return results_df

def group_variable(df, freq = 'hourly'):
    df["fechaobservacion"]= pd.to_datetime(df["fechaobservacion"])
    df["fecha"] = df["fechaobservacion"].dt.date
    df["valorobservado"] = df["valorobservado"].apply(float)
    if freq == 'hourly':
        df["hora"] = df["fechaobservacion"].dt.hour
        df = df.groupby(['codigoestacion', 'codigosensor','nombreestacion', 'departamento', 'municipio', 'zonahidrografica',
       'latitud', 'longitud', 'descripcionsensor', 'unidadmedida', 'fecha',
       'hora']).mean().reset_index() 
        df["fechaobservacion"] = df["fecha"].apply(str)+"T"+df["hora"].apply(str)+":00:00.000"
    elif freq == 'daily':
        df = df.groupby(['codigoestacion', 'codigosensor','nombreestacion', 'departamento', 'municipio', 'zonahidrografica',
       'latitud', 'longitud', 'descripcionsensor', 'unidadmedida', 'fecha']).sum().reset_index() 
        df["fechaobservacion"] = df["fecha"].apply(str)+"T00:00:00.000"
    df = df[['codigoestacion', 'codigosensor', 'fechaobservacion', 'valorobservado',
       'nombreestacion', 'departamento', 'municipio', 'zonahidrografica',
       'latitud', 'longitud', 'descripcionsensor', 'unidadmedida']]
    return df


def API_to_IDEAM_series(df):
    df.columns = ["CodigoEstacion","CodigoSensor","Fecha","Valor","NombreEstacion","Departamento","Municipio", "AreaOperativa","Latitud","Longitud","Etiqueta","Categoria"]
    df["Altitud"] = ""
    df["FechaInstalacion"] = ""
    df["FechaSuspension"]=""
    df["Frecuencia"] = ""
    df["Grado"]=""
    df["NivelAprobacion"] =""
    df["DescripcionSerie"] = ""
    df["Entidad"]=""
    df["IdParametro"] = ""
    df["Calificador"] = ""
    df["CodigoEstacion"] = df["CodigoEstacion"].apply(int)
    df = df[['CodigoEstacion', 'NombreEstacion', 'Latitud', 'Longitud', 'Altitud',
       'Categoria', 'Entidad', 'AreaOperativa', 'Departamento', 'Municipio',
       'FechaInstalacion', 'FechaSuspension', 'IdParametro', 'Etiqueta',
       'DescripcionSerie', 'Frecuencia', 'Fecha', 'Valor', 'Grado',
       'Calificador', 'NivelAprobacion']]
    return df


columns = ["CodigoEstacion","FechaObservacion","ValorObservado","NombreEstacion","ZonaHidrografica","Latitud","Longitud","UnidadMedida"]

In [ ]:
# Example authenticated client (needed for non-public datasets):
client = Socrata('www.datos.gov.co', "mc0EDr4vP5YMKFeiaAEHZUWX0")
Precipitacion = 's54a-sgyg'
Humedad = 'uext-mhny'
Temperatura = 'sbwg-7ju4'
municipality = "CÚCUTA"
municipality_folder = "CUCUTA_IDEAM/"

# PRECIPITATION DATA DOWNLOAD

In [ ]:
prec_stations = get_stations_from_municipality(client, Precipitacion, municipality)
prec_stations

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0016025040,0240,CINERA-VILLA OLGA,8.167777778,-72.46861111,Precipitacion,mm
1,0016015501,0240,APTO CAMILO DAZA,7.932,-72.511,Precipitacion,mm
2,0016027280,0240,ASTILLEROS - AUT,8.105277778,-72.58138889,Precipitacion,mm
3,0016015110,0240,UNIVERSIDAD FRANCISO DE PAULA SANTANDER - AUT,7.898777778,-72.48716667,Precipitacion,mm
4,0016017010,0240,AGUAS CLARAS - AUT,8.233611111,-72.41722222,Precipitacion,mm


In [ ]:
codigos_estaciones = prec_stations["CodigoEstacion"].values
codigos_estaciones = ["0016015501","0016015110","0016025040"]

In [ ]:
prec_time_series = []
for codigo in codigos_estaciones:
    precipitation_data = group_variable(get_time_series_from_station(client, Precipitacion,  municipality, codigoestacion=codigo, limit = 1000000),'daily')
    prec_time_series.append(precipitation_data)

In [ ]:
all_prec_series = pd.concat(prec_time_series,axis = 0).reset_index(drop=True)
all_prec_series.tail()

,codigoestacion,codigosensor,fechaobservacion,valorobservado,nombreestacion,departamento,municipio,zonahidrografica,latitud,longitud,descripcionsensor,unidadmedida
6866,0016025040,0240,2020-06-23T00:00:00.000,0.0,CINERA-VILLA OLGA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,8.167777778,-72.46861111,Precipitacion,mm
6867,0016025040,0240,2020-06-24T00:00:00.000,0.0,CINERA-VILLA OLGA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,8.167777778,-72.46861111,Precipitacion,mm
6868,0016025040,0240,2020-06-25T00:00:00.000,0.0,CINERA-VILLA OLGA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,8.167777778,-72.46861111,Precipitacion,mm
6869,0016025040,0240,2020-06-26T00:00:00.000,1.7,CINERA-VILLA OLGA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,8.167777778,-72.46861111,Precipitacion,mm
6870,0016025040,0240,2020-06-27T00:00:00.000,0.0,CINERA-VILLA OLGA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,8.167777778,-72.46861111,Precipitacion,mm


In [ ]:
all_prec_series = API_to_IDEAM_series(all_prec_series)
all_prec_series.head()

,CodigoEstacion,NombreEstacion,Latitud,Longitud,Altitud,Categoria,Entidad,AreaOperativa,Departamento,Municipio,...,FechaSuspension,IdParametro,Etiqueta,DescripcionSerie,Frecuencia,Fecha,Valor,Grado,Calificador,NivelAprobacion
0,16015501,APTO CAMILO DAZA,7.932,-72.511,,mm,,CATATUMBO,NORTE DE SANTANDER,CÚCUTA,...,,,Precipitacion,,,2015-10-05T00:00:00.000,0.03,,,
1,16015501,APTO CAMILO DAZA,7.932,-72.511,,mm,,CATATUMBO,NORTE DE SANTANDER,CÚCUTA,...,,,Precipitacion,,,2015-10-06T00:00:00.000,1.02,,,
2,16015501,APTO CAMILO DAZA,7.932,-72.511,,mm,,CATATUMBO,NORTE DE SANTANDER,CÚCUTA,...,,,Precipitacion,,,2015-10-07T00:00:00.000,0.54,,,
3,16015501,APTO CAMILO DAZA,7.932,-72.511,,mm,,CATATUMBO,NORTE DE SANTANDER,CÚCUTA,...,,,Precipitacion,,,2015-10-08T00:00:00.000,0.17,,,
4,16015501,APTO CAMILO DAZA,7.932,-72.511,,mm,,CATATUMBO,NORTE DE SANTANDER,CÚCUTA,...,,,Precipitacion,,,2015-10-09T00:00:00.000,0.00,,,


In [ ]:
all_prec_series.to_csv(data_folder+municipality_folder+"/Z_API_precipitacion.csv", index = False)

# TEMPERATURE DATA DOWNLOAD

In [ ]:
temp_stations = get_stations_from_municipality(client, Temperatura, municipality)
temp_stations

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0016015501,0068,APTO CAMILO DAZA,7.932,-72.511,Temp Aire 2 m,°C
1,0016025040,0068,CINERA-VILLA OLGA,8.167777778,-72.46861111,Temp Aire 2 m,°C
2,0016015110,0068,UNIVERSIDAD FRANCISO DE PAULA SANTANDER - AUT,7.898777778,-72.48716667,Temp Aire 2 m,°C


In [ ]:
codigos_estaciones = temp_stations["CodigoEstacion"].values
temp_time_series = []
for codigo in codigos_estaciones:
    temperature_data = group_variable(get_time_series_from_station(client, Temperatura,  municipality, codigoestacion=codigo, limit = 1000000), 'hourly')
    temp_time_series.append(temperature_data)

In [ ]:
all_temp_series = pd.concat(temp_time_series,axis = 0).reset_index(drop=True)
all_temp_series = API_to_IDEAM_series(all_temp_series)
all_temp_series.to_csv(data_folder+municipality_folder+"/Z_API_temperature.csv", index = False)

# HUMEDAD DATA DOWNLOAD

In [ ]:
hum_stations = get_stations_from_municipality(client, Humedad, municipality)
hum_stations

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0016015501,0027,APTO CAMILO DAZA,7.932,-72.511,Humedad del aire 2 mt,%
1,0016025040,0027,CINERA-VILLA OLGA,8.167777778,-72.46861111,Humedad del aire 2 mt,%
2,0016015110,0027,UNIVERSIDAD FRANCISO DE PAULA SANTANDER - AUT,7.898777778,-72.48716667,Humedad del aire 2 mt,%


In [ ]:
codigos_estaciones = hum_stations["CodigoEstacion"].values
hum_time_series = []
for codigo in codigos_estaciones:
    humedad_data = group_variable(get_time_series_from_station(client, Humedad,  municipality, codigoestacion=codigo, limit = 1000000))
    hum_time_series.append(humedad_data)

In [ ]:
hum_time_series = pd.concat(hum_time_series,axis = 0).reset_index(drop=True)
hum_time_series = API_to_IDEAM_series(hum_time_series)
hum_time_series.to_csv(data_folder+municipality_folder+"/Z_API_humidity.csv", index = False)